# Import thư viện cần thiết

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

Đọc dữ liệu

In [2]:
train = pd.read_csv('../data/train_preprocessed.csv')
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [3]:
test = pd.read_csv('../data/test_preprocessed.csv')
test.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,121.00,324


# Xây dựng mô hình học máy

Tổng hợp dữ liệu để xử lý trên các thuộc tính.

In [4]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)

## Loại bỏ các cột chỉ có 1 giá trị độc nhất

Đối với các cột chỉ tồn tại 1 giá trị duy nhất, việc xuất hiện của nó có thể làm cho quá trình dự đoán kết quả trở nên cồng kềnh hơn một cách không cần thiết.

In [5]:
for column in all_data.columns:
    if all_data[column].nunique() == 1:
        all_data.drop(column, axis=1, inplace=True)

## Loại bỏ những thuộc tính dư thừa

### Tạo biến ordinal từ dữ liệu đã được one-hot encode

Các thuộc tính như `epared`, `etecho`, `eviv` có thể được chuyển về dạng dữ liệu ordinal với quy ước **(bad, regular, good) -> (0, 1, 2)** và`instlevel` với giá trị từ **1 - 9**. 

In [6]:
def get_numeric(data, status_name):
    status_cols = [s for s in data.columns.tolist() if status_name in s]
    status_df = data[status_cols]
    status_df.columns = list(range(status_df.shape[1]))
    status_numeric = status_df.idxmax(1)
    status_numeric.name = status_name
    data = pd.concat([data, status_numeric], axis=1)
    return data

In [7]:
status_name_list = ['epared', 'etecho', 'eviv', 'instlevel']
for status_name in status_name_list:
    all_data = get_numeric(all_data, status_name)

### Xóa những thuộc tính không cần thiết

Nhóm nhận thấy có những thuộc tính có thể được xác định bằng những thuộc tính khác trong dữ liệu.

- Nhóm thuộc tính sau với `r4h` và `r4m`:
    ```
    r4t1, persons younger than 12 years of age
    r4t2, persons 12 years of age and older
    r4t3, Total persons in the household
    ```

- Các thuộc tính sau mang cùng ý nghĩa với `hogar_total`:
    ```
    tamhog, size of the household
    tamviv, number of persons living in the household
    hhsize, household size
    r4t3, Total persons in the household
    ```

- `v18q` có thể được tạo ra từ `v18q1`.
- `mobilephone` có thể được tạo ra từ `qmobilephone`.
- `area2` có thể suy ra từ `area1`.
- `female` có thể suy ra từ `male`.
- `epared1~3`, `etecho1~3`, `eviv1~3`, `instlevel1~9` do đã được chuyển đổi thành dữ liệu ordinal nên sẽ không dùng đến nữa.

In [8]:
redundant_features = ['r4t1', 'r4t2', 'r4t3', 
                      'tamhog', 'tamviv', 'hhsize', 'r4t3', 
                      'v18q', 'mobilephone', 'area2', 'female',
                      'epared1', 'epared2', 'epared3', 
                      'etecho1', 'etecho2', 'etecho3',
                      'eviv1', 'eviv2', 'eviv3', 
                      'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']
all_data.drop(columns=redundant_features, inplace=True)

Ngoài ra, nhóm sẽ không loại bỏ các biến bình phương. Vì các biến dữ liệu này thường được biến đổi như một phần của Feature Engineering vì nó có thể giúp các mô hình tuyến tính tìm hiểu các mối quan hệ phi tuyến tính.

## Trích lọc đặc trưng bằng thông số thống kê

Để kết hợp dữ liệu của từng cá nhân vào dữ liệu của cả hộ gia đình, ta cần tổng hợp dữ liệu đó cho từng hộ gia đình. Cách đơn giản nhất để thực hiện việc này là nhóm dữ liệu theo `idhogar` rồi tổng hợp dữ liệu. Tuy nhiên, các dữ liệu **boolean** có thể giống nhau, và sẽ tạo ra nhiều cột dư thừa mà sau đó chúng ta sẽ cần phải loại bỏ sau khi triển khai.

In [9]:
ind_bool = ['dis', 'male',
            'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 
            'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12', 'instlevel']

ind_ordered = ['escolari', 'age']

In [10]:
f = lambda x: x.std(ddof=0)
f.__name__ = 'std_'
ind_agg = all_data.groupby('idhogar')[ind_ordered + ind_bool].agg(['mean', 'max', 'min', 'sum', f])

new_cols = []
for col in ind_agg.columns.levels[0]:
    for stat in ind_agg.columns.levels[1]:
        new_cols.append(f'{col}-{stat}')

ind_agg.columns = new_cols
ind_agg.head()

,escolari-mean,escolari-max,escolari-min,escolari-sum,escolari-std_,age-mean,age-max,age-min,age-sum,age-std_,...,parentesco12-mean,parentesco12-max,parentesco12-min,parentesco12-sum,parentesco12-std_,instlevel-mean,instlevel-max,instlevel-min,instlevel-sum,instlevel-std_
idhogar,,,,,,,,,,,,,,,,,,,,,
000a08204,8.666667,14,0,26,6.182412,20.666667,30,4,62,11.813363,...,0.0,0,0,0,0.0,4.666667,7,0,14,3.299832
000bce7c4,2.500000,5,0,5,2.500000,61.500000,63,60,123,1.500000,...,0.0,0,0,0,0.0,0.500000,1,0,1,0.500000
001845fb0,10.250000,14,6,41,2.861381,35.500000,52,19,142,14.221463,...,0.0,0,0,0,0.0,4.000000,7,2,16,1.870829
001ff74ca,8.000000,16,0,16,8.000000,19.000000,38,0,38,19.000000,...,0.0,0,0,0,0.0,3.500000,7,0,7,3.500000
003123ec2,3.250000,7,0,13,3.269174,12.750000,24,1,51,10.779031,...,0.0,0,0,0,0.0,1.250000,3,0,5,1.299038


In [11]:
corr_matrix = ind_agg.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any((abs(upper[column]) > 0.95) & (abs(upper[column]) == 1))]
print(f'There are {len(to_drop)} correlated columns to remove')

There are 2 correlated columns to remove


In [12]:
all_data = all_data.merge(ind_agg, on = 'idhogar', how = 'left')
all_data.drop(columns=ind_bool+ind_ordered+to_drop, inplace=True)
print('Number of features after dropping the individual level features', all_data.shape[1])

Number of features after dropping the individual level features 213


## Tạo thêm cột phụ biểu hiện thêm thông tin của hộ gia đình

Trong một hộ gia đình, cần biết rằng người trưởng thành có khả năng làm việc là trụ cột của gia đình, trẻ em và người lớn trên 65 tuổi thì không làm việc. Do đó việc quyết định việc xem xét đánh giá tình trạng của một gia đình không phụ thuộc nhiều vào tỉ lệ các nhóm đối tượng trong hộ gia đình. Vì vậy ta sẽ tạo thêm một số cột liên quan đến vấn đề này.

In [13]:
def extract_features(df):
    df['adult_num'] = df['hogar_adul'] - df['hogar_mayor'] # số lượng người trưởng thành còn khả năng làm việc
    df['head_is_adult'] = (df['adult_num'] > 0).astype(int) # có người trưởng thành là trụ cột
    df['adult_rate'] = df['adult_num'] / df['hogar_total'] # tỉ lệ người trưởng thành
    
    df['dependency_num'] = df['hogar_nin'] + df['hogar_mayor'] # số người phụ thuộc
    df['dependency_rate'] = df['dependency_num'] / df['hogar_total'] # tỉ lệ phụ thuộc
    
    df['adult_dependency_rate'] = df['adult_num'] / (df['dependency_num'] + 0.1) # tỉ lệ người trưởng thành trên người phụ thuọc
    df['children_rate'] = df['hogar_nin'] / df['hogar_total'] # tỉ lệ trẻ em trong gia đình
    df['elder_rate'] = df['hogar_mayor'] / df['hogar_total'] # tỉ lệ người già trong gia đình

    df['rent_per_person'] = df['v2a1'] / df['hogar_total'] # giá thuê nhà tính trên mỗi người
    df['rent_per_adult'] = df['v2a1'] / (df['adult_num'] + 0.1) # gía thuê nhà tính trên người thưởng thành

    df['bedroom_per_person'] = df['bedrooms'] / df['hogar_total'] # số lượng người trung bình mỗi phòng
    df['bedroom_per_adult'] = df['bedrooms'] / (df['adult_num'] + 0.1) # số lượng người trung bình mỗi phòng cho người trưởng thành
    
extract_features(all_data)

## Feature scaling

Đối với cột có giá trị lớn (từ 100-100000), nhóm sẽ dùng `Standard Scaler` để đưa dữ liệu về phân phối chuẩn.

Đối với các cột còn lại, nhóm sẽ dùng `MinMax Scaler` để đưa giá trị về khoảng (0,1).

In [14]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

col_std = ['v2a1', 'rent_per_person', 'rent_per_adult']
col_minmax = list(set(all_data.drop(columns=['Id', 'idhogar', 'Target']).columns) - set(col_std))

std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

all_data[col_std] = std_scaler.fit_transform(all_data[col_std])
all_data[col_minmax] = minmax_scaler.fit_transform(all_data[col_minmax])

## Triển khai mô hình

### Chuẩn bị dữ liệu

In [15]:
train = all_data[:ntrain][:]
test = all_data[ntrain:][:]

Lấy dữ liệu dùng để huấn luyện model từ tập `train`.

In [16]:
X = train.drop(columns=['Id', 'idhogar', 'Target'])
y = train.Target.to_numpy().astype('int') - 1

Lấy dữ liệu dùng để huấn luyện từ tập `test`.

In [17]:
data_test_id = list(test.Id)
data_test = test.drop(columns=['Id', 'idhogar', 'Target'])

### Xử lý mất cân bằng dữ liệu

In [18]:
print('Tỉ lệ các label để train ',
      train['Target'].value_counts()/train.shape[0])

Tỉ lệ các label để train  Target
4.0    0.627394
2.0    0.167103
3.0    0.126504
1.0    0.079000
Name: count, dtype: float64


Theo tỉ lệ trên có thể thấy được label 4 gấp 10 lần label 1. Nếu không
xử lý có thể làm mô hình dự đoán không hiệu quả.

**Cách xử lý:** Gán cho mỗi label một trọng số. Trọng số này tỉ lệ nghịch với
số lượng label. Dữ liệu nào ít nhãn thì có trọng số cao.

In [19]:
from sklearn.utils import class_weight

y_weights = class_weight.compute_sample_weight('balanced', y, indices=None)
print(pd.DataFrame(dict(Target = y, Weight = y_weights)).drop_duplicates().sort_values(by = ["Target"]).reset_index(drop = True))

   Target    Weight
0       0  3.164570
1       1  1.496086
2       2  1.976220
3       3  0.398474


Bằng cách này, mô hình chú ý nhiều hơn đến label thiểu số và đưa ra
những dự đoán tốt hơn cho label đó.

Chia dữ liệu huấn luyện thành tập train và test.

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test, weights_train,weights_test = train_test_split(X,y,y_weights,test_size=0.2, random_state=1)

### Xây dựng mô hình

In [21]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

#### K-Nearest Neighbors Classifier

In [22]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors=4)

model_knn.fit(X_train,y_train)

print(classification_report(y_test, model_knn.predict(X_test),target_names=['class 1', 'class 2', 'class 3','class 4']))
print("F1 score macro: ", f1_score(y_test, model_knn.predict(X_test), average = 'macro'))

              precision    recall  f1-score   support

     class 1       0.74      0.78      0.76       147
     class 2       0.74      0.75      0.75       333
     class 3       0.68      0.69      0.68       229
     class 4       0.90      0.89      0.90      1203

    accuracy                           0.83      1912
   macro avg       0.76      0.78      0.77      1912
weighted avg       0.84      0.83      0.83      1912

F1 score macro:  0.7712625363305321


#### Random Forest Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(n_estimators=100,
                                     criterion='log_loss',
                                     max_features=None,
                                     class_weight='balanced_subsample',)

model_rfc.fit(X_train, y_train, **{'sample_weight': weights_train})

print(classification_report(y_test, model_rfc.predict(X_test), target_names=['class 1', 'class 2', 'class 3', 'class 4']))
print("F1 score macro: ", f1_score(y_test, model_rfc.predict(X_test), average = 'macro'))

              precision    recall  f1-score   support

     class 1       0.95      0.88      0.92       147
     class 2       0.94      0.92      0.93       333
     class 3       0.94      0.89      0.91       229
     class 4       0.96      0.99      0.98      1203

    accuracy                           0.96      1912
   macro avg       0.95      0.92      0.93      1912
weighted avg       0.96      0.96      0.96      1912

F1 score macro:  0.9335900808466069


#### Gradient Boosting Classifier

In [25]:
from sklearn.ensemble import GradientBoostingClassifier\

model_gbc = GradientBoostingClassifier()

best_param =  {'learning_rate': 0.3, 
               'max_depth': 7, 
               'n_estimators': 200}


model_gbc.set_params(**best_param) # set siêu tham số cho mô hình

model_gbc.fit(X_train, y_train, **{'sample_weight': weights_train})

print(classification_report(y_test, model_gbc.predict(X_test),target_names=['class 1', 'class 2', 'class 3','class 4']))
print("F1 score macro: ", f1_score(y_test, model_gbc.predict(X_test), average = 'macro'))

              precision    recall  f1-score   support

     class 1       0.93      0.89      0.91       147
     class 2       0.94      0.91      0.93       333
     class 3       0.92      0.91      0.91       229
     class 4       0.97      0.99      0.98      1203

    accuracy                           0.96      1912
   macro avg       0.94      0.93      0.93      1912
weighted avg       0.96      0.96      0.96      1912

F1 score macro:  0.932901778755064


#### XGBoost Classifier

In [26]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(
    booster='gbtree',
    learning_rate=0.1,
    n_estimators=1200,
    max_depth=6,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    num_class=4,
    reg_alpha=0,
    reg_lambda=1,
    random_state=42,
    verbosity=0
) 

model_xgb.fit(X_train, y_train)

print(classification_report(y_test, model_xgb.predict(X_test), target_names=['class 1', 'class 2', 'class 3', 'class 4']))
print("F1 score macro: ", f1_score(y_test, model_xgb.predict(X_test), average = 'macro'))

              precision    recall  f1-score   support

     class 1       0.91      0.88      0.89       147
     class 2       0.94      0.92      0.93       333
     class 3       0.92      0.90      0.91       229
     class 4       0.97      0.99      0.98      1203

    accuracy                           0.96      1912
   macro avg       0.94      0.92      0.93      1912
weighted avg       0.96      0.96      0.96      1912

F1 score macro:  0.9273183963390006


#### Light GBM

In [27]:
import lightgbm as lgb

model_lgbm = lgb.LGBMClassifier(class_weight='balanced', boosting_type='dart',
                                drop_rate=0.9, min_data_in_leaf=100, 
                                max_bin=255,
                                n_estimators=500,
                                bagging_fraction=0.01,
                                min_sum_hessian_in_leaf=1,
                                importance_type='gain',
                                learning_rate=0.1, 
                                max_depth=-1, 
                                num_leaves=31,
                                verbose=-1)

model_lgbm = model_lgbm.fit(X_train, y_train)

print(classification_report(y_test, model_lgbm.predict(X_test),target_names=['class 1', 'class 2', 'class 3','class 4']))
print("F1 score macro: ", f1_score(y_test, model_lgbm.predict(X_test), average = 'macro'))


              precision    recall  f1-score   support

     class 1       0.87      0.88      0.88       147
     class 2       0.92      0.92      0.92       333
     class 3       0.87      0.90      0.89       229
     class 4       0.98      0.97      0.97      1203

    accuracy                           0.95      1912
   macro avg       0.91      0.92      0.91      1912
weighted avg       0.95      0.95      0.95      1912

F1 score macro:  0.9143556147897771


### Submit

In [28]:
knn_submission = pd.DataFrame({'Id': data_test_id, 'Target': model_knn.predict(data_test)+1})
knn_submission.to_csv('submission_knn.csv', index = False)

In [29]:
rfc_submission = pd.DataFrame({'Id': data_test_id, 'Target': model_rfc.predict(data_test)+1})
rfc_submission.to_csv('submission_rfc.csv', index = False)

In [30]:
gbc_submission = pd.DataFrame({'Id': data_test_id, 'Target': model_gbc.predict(data_test)+1})
gbc_submission.to_csv('submission_gbc.csv', index = False)

In [31]:
xgb_submission = pd.DataFrame({'Id': data_test_id, 'Target': model_xgb.predict(data_test)+1})
xgb_submission.to_csv('submission_xgb.csv', index = False)

In [32]:
lgbm_submission = pd.DataFrame({'Id': data_test_id, 'Target': model_lgbm.predict(data_test)+1})
lgbm_submission.to_csv('submission_lgb.csv', index = False)